In [49]:
import numpy as np
import biotuner
from biotuner.biotuner_utils import *
from biotuner.biotuner_object import *
from biotuner.dictionaries import *
import matplotlib.pyplot as plt
import pytuning.visualizations.scales
from pytuning.visualizations.scales import consonance_matrix
import csv
import os
import time
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

This notebook is intended to demonstrate the use of the biotuner (https://github.com/antoineBellemare/biotuner/)

The core idea is to extract harmonic information from biological time series to inform the computation of musical structures

The terms ''tuning'' and ''scale'' will be used interchangeably, referring to a series of ratios that subdivides an octave.

## Load dataset

In [50]:
data = np.load('data_examples/EEG_pareidolia/parei_data_1000ts.npy')

## Using the peaks_extraction method

There is no minimum size for time series. However, keep in mind that the minimum frequency that can be observed using spectral decomposition is dependant on the length of your time series. for example, if the sampling frequency equals 1000Hz, time series shorter than 1 second will not allow for peak extraction below 2Hz.

Peaks functions (link to doc):

'fixed' : ranges of frequency bands are fixed

'adapt' : ranges of frequency bands are defined based on the alpha peak

'EMD': Intrinsic Mode Functions (IMFs) are derived from Empirical Mode Decomposition (EMD)  
                   FFT is computed on each IMF
                   
'EEMD': Intrinsic Mode Functions (IMFs) are derived from Ensemble Empirical Mode Decomposition (EMD)  
                    FFT is computed on each IMF
                    
'HH1D_max': maximum values of the 1d Hilbert-Huang transform on each IMF using EEMD.

'harmonic_recurrence': keeps peaks for which a maximum of other peaks are harmonics

'cepstrum': peak frequencies of the cepstrum (inverse Fourier transform (IFT) of the logarithm of the estimated signal spectrum)

'FOOOF' : peaks rising above the aperiodic component

In [51]:
data_ = data[28]
start = time.time()
FREQ_BANDS = [[1, 3], [3, 7], [7, 12], [12, 18], [18, 30], [30, 45]] # Define frequency bands for peaks_function = 'fixed'

biotuning = compute_biotuner(sf = 1000, peaks_function = 'fixed', precision = 0.5, n_harm = 10,
                    ratios_n_harms = 5, ratios_inc_fit = True, ratios_inc = True) # Initialize biotuner object

biotuning.peaks_extraction(data_, FREQ_BANDS = FREQ_BANDS, ratios_extension = True, max_freq = 30, n_peaks=5,
                          graph=False, min_harms=2)
biotuning.compute_peaks_metrics()

stop = time.time()
print(stop-start)

Index_max: all zeros indicate 1/f trend [0, 7, 2, 11, 1, 17]
Number of peaks : 6
2.948944091796875


In [52]:
# Print the extracted peaks
biotuning.peaks

array([ 1. ,  6.5,  8. , 17.5, 18.5, 38.5])

## Constructing euclidian rhythms from biotunings

Euclidian rhythms represent an even distribution of pulses within a specified number of steps. It is possible to convert frequency ratios into euclidian rhythms in that a ratio's numerator can be seen as the number of pulses and a ratio's denominator as the number of steps within which the pulses should be distributed. 

In [ ]:
# Initialize biotuner object
biotuning_harm_peaks = compute_biotuner(sf = 1000, peaks_function = 'harmonic_recurrence', precision = 0.5) 

# Extract spectral peaks
biotuning_harm_peaks.peaks_extraction(data_, min_freq = 5, max_freq = 20, min_harms = 2, harm_limit = 128)
print(biotuning_harm_peaks.all_harmonics)

biotuning_harm_peaks.peaks_extraction(data_, min_freq = 5, max_freq = 20, min_harms = 4, harm_limit = 128)
print(biotuning_harm_peaks.all_harmonics)

In [ ]:
harm_tuning = harmonic_tuning(biotuning_harm_peaks.all_harmonics)
harm_tuning

In [88]:
cons_harm_tuning = sort_scale_by_consonance(harm_tuning) # sorting a tuning by consonance of intervals
cons_harm_tuning

NameError: name 'sort_scale_by_consonance' is not defined

'scale2euclid' function derives euclidian rhythms from a series of ratios.

When mode = 'normal', the ratios are inverted and the numerator is taken as the number of pulses and the denominator as the number of steps (e.g. 3/2 -> 2pulses/3steps).

When mode = 'full', the ratios are inverted. The number of steps corresponds to num*denom and the numbers of pulses correspond to both the initial numerator and denominator (e.g. 3/2 -> 3pulses/6steps and 2pulses/6steps)

In [ ]:
euclid_patterns = scale2euclid(cons_harm_tuning[2:8], max_denom = 16, mode = 'normal')
harm_tuning_frac, _, _ = scale2frac(cons_harm_tuning[2:8])
interval_vectors = [interval_vector(x) for x in euclid_patterns]
harm_tuning_frac, euclid_patterns, interval_vectors

([5/4, 7/4, 9/8, 13/8, 15/8, 17/16],
 [[1, 1, 1, 1, 0],
  [1, 0, 1, 0, 1, 0, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 0],
  [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]],
 [[1, 1, 1, 2],
  [2, 2, 2, 1],
  [1, 1, 1, 1, 1, 1, 1, 2],
  [2, 1, 2, 2, 1, 2, 2, 1],
  [2, 2, 2, 2, 2, 2, 2, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]])

'consonant_euclid' function derives euclidian rhythm from a set of consonant intervals. 

First, a set of denominators is defined by finding the common denominator of each pair of ratios (e.g. 3/5 and 4/7 give 5*7=35). 

The consonance is computed on each pairs of denominators to find harmonic subdivisions. Only the euclidian rhythms from consonant denominators (steps) are returned.

In [ ]:
euclid_final, cons = consonant_euclid(harm_tuning2, n_steps_down = 2, limit_denom = 8, 
                                      limit_cons = 0.1, limit_denom_final = 16)
interval_vectors = [interval_vector(x) for x in euclid_final]
strings = interval_vec_to_string(interval_vectors)
euclid_referent = euclid_string_to_referent(strings, dict_rhythms)
euclid_final, interval_vectors, euclid_referent

C:\Users\Dell\Anaconda3\envs\biotuner\lib\fractions.py:597: RuntimeWarning: overflow encountered in longlong_scalars
  self._denominator * other.numerator)


([[1, 0],
  [1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0, 1, 0, 0, 0],
  [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
  [1, 0, 0, 1, 0, 0],
  [1, 0, 0, 1, 0, 0, 0],
  [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
  [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
  [1, 0, 1, 0, 1, 0],
  [1, 0, 1, 0, 1, 0, 0],
  [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0],
  [1, 0, 1, 0, 1, 0, 1, 0],
  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
  [1, 0, 1, 1, 0, 1, 1, 0],
  [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0],
  [1, 1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0]],
 [[2],
  [3],
  [4],
  [4, 4],
  [4, 4, 4],
  [3, 3],
  [3, 4],
  [3, 4, 3, 4],
  [3, 3, 3, 3],
  [2, 2, 2],
  [2, 2, 3],
  [2, 2, 3, 2, 2, 3],
  [2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2, 2],
  [2, 1, 2, 1, 2],
  [2, 1, 2, 1, 2, 2, 1, 2, 1, 2],
  [1, 1, 1, 1, 1, 2],
  [1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2]],
 ['None',
  'None',
  'None',
  'None',
  'None',
  'None',


In [ ]:
interval_vector(bjorklund(16, 13))

[1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1]